In [ ]:
import ray
ray.init()

In [ ]:
resources = ray.cluster_resources()
print(resources)

In [ ]:
#env = gym.make('FrankaPickPlaceFixed-v0')

In [ ]:
def env_creator(env_config={}):
    import gymnasium as gym
    env = gym.make('FetchPickAndPlace-v2', render_mode='rgb_array', reward_type='dense', max_episode_steps=100)
    #env = CustomWrapper(env)
    env.reset()
    return env

In [ ]:
from ray.tune.registry import register_env
register_env("Gymnasium_Pick_Place_0_Dense", env_creator)

In [ ]:
# Serialize with cloudpickle and save to a file
#with open("Gymnasium_Pick_Place_0.pkl", "wb") as f:
   # cloudpickle.dump(env_creator, f)

In [ ]:
from ray import tune

In [ ]:
tune.run("PPO",
             # algorithm specific configuration
             config={"env": "Gymnasium_Pick_Place_0_Dense",  #
                     "framework": "torch",
                     #"seed":958,
                     "num_gpus": 1,
                     "num_rollout_workers": 1,
                     #"num_envs_per_worker": 1,
                     #"num_gpus_per_worker": 0,
                     "train_batch_size": 512,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 32,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.00015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2,
                     "evaluation_parallel_to_training": True,
                     "evaluation_num_workers": 1
                     },
             local_dir="../R3L-LOGS",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"training_iteration": 5000}
             )

In [ ]:
import gymnasium as gym
env = gym.make('FetchPickAndPlace-v2', render_mode='rgb_array', reward_type='dense', max_episode_steps=100)
#env = CustomWrapper(env)
env.reset()

In [ ]:
import numpy as np

from ray.rllib.policy.policy import Policy     

# Use the `from_checkpoint` utility of the Policy class:
my_restored_policy = Policy.from_checkpoint("../R3L-LOGS/PPO/PPO_Gymnasium_Pick_Place_0_Dense_df6f8_00000_0_2023-07-28_10-19-37/checkpoint_004994/policies/default_policy")

# Use the restored policy for serving actions.
obs,  reward, terminated, truncated, info =   env.step(env.action_space.sample())# individual CartPole observation
#action = my_restored_policy.compute_single_action(np.append(obs.get('observation'), obs.get('achieved_goal'), obs.get('desired_goal')))


In [ ]:
for i in range(0, 512):
    action = my_restored_policy.compute_single_action(np.concatenate([obs.get('observation'), obs.get('desired_goal'), obs.get('achieved_goal')]))

    obs,  reward, terminated, truncated, info =   env.step(env.action_space.sample())# individual CartPole observation

    print(f"Computed action {action} from given CartPole observation.")

In [ ]:
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
img = plt.imshow(env.render())
for i in range(0, 512):
    
    action = my_restored_policy.compute_single_action(np.concatenate([obs.get('observation'), obs.get('desired_goal'), obs.get('achieved_goal')]))

    obs,  reward, terminated, truncated, info =   env.step(env.action_space.sample())# individual CartPole observation

    #print(f"Computed action {action} from given CartPole observation.")
    img.set_data(env.render()) # Just update the data
    ipythondisplay.display(plt.gcf())
    ipythondisplay.clear_output(wait=True)

In [ ]:
#import ray
import gymnasium as gym
from ray.rllib.algorithms.ppo import PPO

# Load the trained model.
#ray.init()

config = {
    "env": "Gymnasium_Pick_Place_0", 
    # add any additional configuration settings here
}
agent = PPO(config=config)
agent.restore("/home/cocp5/Gymnasium-Robotics-RL/Gymnasium_Pick_Place_0/PPO/PPO_Gymnasium_Pick_Place_0_d144a_00000_0_2023-07-25_15-52-03/checkpoint_000874")



In [ ]:
env.render()

In [ ]:
agent.evaluate()

In [ ]:
# Create the environment
env = gym.make('FetchPickAndPlace-v2', max_episode_steps=100)#(config["env"])

# Run the agent on the environment for 10 episodes.
for i_episode in range(10):
    observation, info = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action = agent.compute_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        env.render()
    print(f"Episode {i_episode + 1} reward: {episode_reward}")

In [ ]:
#import ray
from ray.rllib.algorithms.ppo import PPOTrainer, DEFAULT_CONFIG

# Initialize Ray.
ray.init()

# Define the configuration.
config = DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1

# Create the trainer.
trainer = PPOTrainer(config=config, env="Gymnasium_Pick_Place_0")

# Load the trained model.
trainer.restore("/path/to/the/saved/checkpoint")

# Run the agent on the environment for 10 episodes.
for i_episode in range(10):
    episode_reward = 0
    done = False
    obs = trainer.env.reset()
    while not done:
        action = trainer.compute_action(obs)
        obs, reward, done, info = trainer.env.step(action)
        episode_reward += reward
        trainer.env.render()
    print(f"Episode {i_episode + 1} reward: {episode_reward}")
